In [1]:
import json

notebook_file = "/home/ryounis/Documents/Zurich/PEACHLab/datascience-visualisation/data/datasets/Natural Language Processing with Disaster Tweets/classified_notebooks.json"
with open(notebook_file, 'r') as file:
    data = json.load(file)

In [ ]:
import os
from Clusterers.clusterer import ClassCluster
from tqdm import tqdm

with open('../../../secrets/api_key.txt', 'r') as f: api_key = f'{f.read()}'

os.environ["OPENAI_API_KEY"] = api_key
clusterer = ClassCluster()
for notebook in tqdm(data["notebooks"]):
    for cell in notebook["cells"]:
        cell["embedding"] = clusterer.embed_cell(cell["code"], cell["desc"])

In [ ]:
import os
from tqdm import tqdm
import sys; sys.path.insert(0, "../")
from Clusterers.clusterer import ClassCluster
with open('../../../secrets/api_key.txt', 'r') as f: api_key = f'{f.read()}'

os.environ["OPENAI_API_KEY"] = api_key
clusterer = ClassCluster()
for notebook in tqdm(data["notebooks"]):
    for cell in notebook["cells"]["cells"]:
        cell["embedding"] = clusterer.embed_cell(cell["code"], cell["desc"])

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

embeddings = np.array([cell["embedding"] for notebook in data["notebooks"] for cell in notebook["cells"]])

pca = PCA(n_components=2)
pca.fit(embeddings)
embeddings_pca = pca.transform(embeddings)
principal_components = pca.components_
explained_variance_ratio = pca.explained_variance_ratio_

In [ ]:
import pandas as pd
from pandas.plotting import scatter_matrix

import matplotlib.pyplot as plt

# Convert the embeddings_pca array to a DataFrame
df_embeddings_pca = pd.DataFrame(embeddings_pca, columns=['Principal Component 1', 'Principal Component 2'])

# Create the scatter matrix
scatter_matrix(df_embeddings_pca, figsize=(8, 8), alpha=0.5)
plt.show()